In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()


In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=',', header=True)


In [ ]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')
spark.sql('select * from home_sales').show()

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql('select year(date) as year, round(avg(price), 2) as AVERAGE_PRICE from home_sales where bedrooms = 4 group by year(date)').show()

In [ ]:
# 4. What is the average price of a home for each year the home was built
# that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql('select year(date) as year, round(avg(price)) as AVERGE_PRICE from home_sales where bedrooms = 3 and bathrooms = 3 group by year(date)').show()

In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql('select year(date) as year, round(avg(price), 2) as AVERGE_PRICE from home_sales where bedrooms = 3 and bathrooms = 3 and floors = 2 and round(sqft_living , 2) >= 2000 group by year(date)').show()

In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()

spark.sql('select view, round(avg(price), 2) as AVERAGE_PRICE from home_sales where price >= 350000 group by view').show()


print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached("home_sales")

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql('select view, round(avg(price), 2) as AVERAGE_PRICE from home_sales where price <= 350000 group by view').show()

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 10. Partition the home sales dataset by date_built field.
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("sales_partitioned")

In [ ]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('sales_partitioned')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('sales_partitioned')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql('select view, round(avg(price), 2) as AVERAGE_PRICE from sales_partitioned where price <= 350000 group by view').show()

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached('home_sales'):
  print('home_sales is still cached')
else:
  print('all clear')
